Steps for building the model
- Importing and understanding data
- Preparing data
  - imputation if any
  - dummy variables
  - split data into test and train sets
  - rescaling
- Building model
  - start with recursive feature elimination (RFE)
  - build model using statsmodel with RFE features
  - optimize model (drop features if insignificant)
- Residual analysis
- Prediction on the test set
- Evaluate Model

In [1]:
# import libraries
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')


In [2]:
# read data
df = pd.read_csv('./day.csv')

In [7]:
# skim through data set
df

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,27-12-2019,1,1,12,0,4,1,2,10.420847,11.33210,65.2917,23.458911,247,1867,2114
726,727,28-12-2019,1,1,12,0,5,1,2,10.386653,12.75230,59.0000,10.416557,644,2451,3095
727,728,29-12-2019,1,1,12,0,6,0,2,10.386653,12.12000,75.2917,8.333661,159,1182,1341
728,729,30-12-2019,1,1,12,0,0,0,1,10.489153,11.58500,48.3333,23.500518,364,1432,1796


In [13]:
df.shape

(730, 16)

In [12]:
# how are the days with max and min sales
df.sort_values(by='cnt',ascending=False)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
622,623,15-09-2019,3,1,9,0,6,0,1,24.941653,29.29335,50.1667,16.583907,3160,5554,8714
636,637,29-09-2019,4,1,9,0,6,0,1,22.242500,26.51480,54.2917,15.249468,2589,5966,8555
629,630,22-09-2019,3,1,9,0,6,0,1,26.650000,30.52440,64.6667,19.000061,2512,5883,8395
446,447,23-03-2019,2,1,3,0,5,1,2,24.668347,28.50335,69.4167,7.792100,2469,5893,8362
503,504,19-05-2019,2,1,5,0,6,0,1,24.600000,28.34540,45.6250,5.626325,3410,4884,8294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,06-03-2018,1,0,3,0,0,0,2,15.437402,18.31260,94.8261,23.000229,114,491,605
25,26,26-01-2018,1,0,1,0,3,1,3,8.917500,10.18000,86.2500,19.687950,34,472,506
724,725,26-12-2019,1,1,12,0,3,1,3,9.976653,11.01665,82.3333,21.208582,9,432,441
26,27,27-01-2018,1,0,1,0,4,1,1,7.995000,10.98500,68.7500,7.627079,15,416,431


Naturally, looks like non working days with clear weather conditions have had the best sales,
and working days with non-clear weather conditions have had the least sales.

Season wise - summer and fall had high sales, winter and spring had low.